In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import sys
sys.path.append("../..")
import evoten
from Bio import SeqIO
import numpy as np

In [2]:
msa_file = "PF00008.fasta"
tree_file = "PF00008.fasta.raxml.bestTree"
alphabet = "ARNDCQEGHILKMFPSTWYV."

In [3]:
msa_file = "PF00028.fasta"
tree_file = "PF00028.fasta.raxml.bestTree"
alphabet = "ARNDCQEGHILKMFPSTWYV."

In [4]:
# parse fasta
import evoten.model

leaf_names = []
seqs = []
for record in SeqIO.parse(msa_file, "fasta"):
    leaf_names.append(record.id)
    seqs.append(str(record.seq))

one_hot_leaves = evoten.util.encode_one_hot(seqs, alphabet=alphabet)
one_hot_leaves = one_hot_leaves[:, np.newaxis] # add model dimension

# create tree
tree = evoten.TreeHandler.read(tree_file)

In [5]:
len(seqs), tree.height

(55, 12)

In [6]:
num_models = 10

tree.set_branch_lengths(np.repeat(tree.branch_lengths, num_models, -1))

### TensorFlow

In [7]:
evoten.set_backend("tensorflow")

2024-12-05 10:06:48.149330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 10:06:48.165392: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 10:06:48.170292: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 10:06:48.182967: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-05 10:06:48.866927: W tensorflow/compiler/tf2

In [8]:
R, pi = evoten.substitution_models.jukes_cantor(d = len(alphabet))
rate_matrix = evoten.model.backend.make_rate_matrix(R, pi)

2024-12-05 10:06:49.428769: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-12-05 10:06:49.428796: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: powergreif
2024-12-05 10:06:49.428801: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: powergreif
2024-12-05 10:06:49.428867: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 535.183.1
2024-12-05 10:06:49.428885: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] kernel reported version is: 535.183.1
2024-12-05 10:06:49.428890: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:249] kernel version seems to match DSO: 535.183.1


In [9]:
#compile tf function once
L = evoten.model.loglik(one_hot_leaves, leaf_names, tree, rate_matrix, tree.branch_lengths, pi,
                            leaves_are_probabilities=True)

In [10]:
%%timeit -n 100
L = evoten.model.loglik(one_hot_leaves, leaf_names, tree, rate_matrix, tree.branch_lengths, pi,
                            leaves_are_probabilities=True)

114 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Pytorch

In [11]:
evoten.set_backend("pytorch")

In [12]:
R, pi = evoten.substitution_models.jukes_cantor(d = len(alphabet))
rate_matrix = evoten.model.backend.make_rate_matrix(R, pi)

In [ ]:
%%timeit -n 100
L = evoten.model.loglik(one_hot_leaves, leaf_names, tree, rate_matrix, tree.branch_lengths, pi,
                            leaves_are_probabilities=True)

27.8 ms ± 235 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
